Import the libraries 

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

Loading the data and splitting it into features and targets. 

1. We then set the train_size to 80% of the size of training data and we perform the test, train split

In [67]:
# Load the data
data = pd.read_csv("/content/data/yield_data.csv")

# Split the data into features and targets
X = data[["Temperature (°C)"	,"Humidity (%)",	"Rainfall (mm)"]]
y = data[["Apple Yield (tons)",	"Orange Yield (tons)"]]

# Split the data into training and testing sets
train_size = int(0.8 * len(data))
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

We then visualize the data 

In [3]:
data.head()

,Temperature (°C),Humidity (%),Rainfall (mm),Apple Yield (tons),Orange Yield (tons)
0,18.595528,52.578431,86.207877,41.672519,36.470067
1,19.746958,75.403745,23.143332,45.647457,37.289553
2,26.087425,75.858556,57.170663,44.190152,14.861612
3,22.079828,73.673775,98.237110,31.475872,11.865185
4,23.983127,81.669593,60.065625,43.271421,31.353973


In [4]:
X.head(), X_train.head(), X_test.head(), y.head(), y_train.head(), y_test.head()

(   Temperature (°C)  Humidity (%)  Rainfall (mm)
 0         18.595528     52.578431      86.207877
 1         19.746958     75.403745      23.143332
 2         26.087425     75.858556      57.170663
 3         22.079828     73.673775      98.237110
 4         23.983127     81.669593      60.065625,
    Temperature (°C)  Humidity (%)  Rainfall (mm)
 0         18.595528     52.578431      86.207877
 1         19.746958     75.403745      23.143332
 2         26.087425     75.858556      57.170663
 3         22.079828     73.673775      98.237110
 4         23.983127     81.669593      60.065625,
      Temperature (°C)  Humidity (%)  Rainfall (mm)
 800         17.807829     75.402491      39.437818
 801         15.790894     59.793964      61.583397
 802         22.642642     65.394859      43.320417
 803         26.643526     52.079583      83.875216
 804         26.923264     54.880430      26.256636,
    Apple Yield (tons)  Orange Yield (tons)
 0           41.672519            36.4700

In [5]:
len(X_train), len(X_test), len(y_train), len(y_test)

(800, 200, 800, 200)

We construct out MultivariateLinearRegression Model by subclassing the nn.Module 

1. In total there are 10 layers 
> The first layer is the input layer 
>layers 2 to 9 are the hidden layers
2. layer 10 is the output layer. 

our forward pass employs the rectified linear unit (Relu) activation function to each layer. 

We return the out features. 

In [49]:
class MultiVariateLinearRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.layer1 = nn.Linear(input_size, 64)
        self.layer2 = nn.Linear(64, 128)
        self.layer3 = nn.Linear(128, 256)
        self.layer4 = nn.Linear(256, 512)
        self.layer5 = nn.Linear(512, 256)
        self.layer6 = nn.Linear(256, 128)
        self.layer7 = nn.Linear(128, 64)
        self.layer8 = nn.Linear(64, 32)
        self.layer9 = nn.Linear(32, 16)
        self.layer10 = nn.Linear(16, output_size)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.relu(self.layer3(x))
        x = torch.relu(self.layer4(x))
        x = torch.relu(self.layer5(x))
        x = torch.relu(self.layer6(x))
        x = torch.relu(self.layer7(x))
        x = torch.relu(self.layer8(x))
        x = torch.relu(self.layer9(x))
        out = self.layer10(x)
        return out


Here we set out hyper parameters (done through experimenting with different values 

1. The input size is 3 as there are 3 features 
2. The output-size is 2 as there are 2 targets 

In [32]:
# Define the hyperparameters
learning_rate = 0.001
num_epochs = 1000
input_size = 3
output_size = 2

We create the model and also add a loss function (MAE loss).

To update the model parameters we use the Adam optimizer.

In [50]:
# Initialize the model, loss function, and optimizer
model = MultiVariateLinearRegression(input_size, 
                                     output_size)
lossfn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), 
                            lr=learning_rate)

In [68]:
model.state_dict

<bound method Module.state_dict of MultiVariateLinearRegression(
  (layer1): Linear(in_features=3, out_features=64, bias=True)
  (layer2): Linear(in_features=64, out_features=128, bias=True)
  (layer3): Linear(in_features=128, out_features=256, bias=True)
  (layer4): Linear(in_features=256, out_features=512, bias=True)
  (layer5): Linear(in_features=512, out_features=256, bias=True)
  (layer6): Linear(in_features=256, out_features=128, bias=True)
  (layer7): Linear(in_features=128, out_features=64, bias=True)
  (layer8): Linear(in_features=64, out_features=32, bias=True)
  (layer9): Linear(in_features=32, out_features=16, bias=True)
  (layer10): Linear(in_features=16, out_features=2, bias=True)
)>

We first convert data frame to numpy array.

In [46]:
a = X_train.values 
b = y_train.values 
c = X_test.values 
d = y_test.values 

The basic work flow is 
1. clear the gradients else they get accummulated in pytorch 
2. pass the input to the model 
3. calculate the loss 
4. calculate the gradients using loss.backward()
5. Then perform the optimization. 

Repeat for epochs

In [51]:
def train(model, criterion, optimizer, X_train, y_train):
    # Convert numpy arrays to tensors
    inputs = torch.from_numpy(a).float()
    labels = torch.from_numpy(b).float()

    # Zero the parameter gradients
    optimizer.zero_grad()

    # Forward + backward + optimize
    outputs = model(inputs)
    loss = lossfn(outputs, labels)
    loss.backward()
    optimizer.step()

    return loss.item()


For each epoch calculate the loss and percent loss, 

keep track of it to see how the model performs 

update the hyper parameters if necessary

In [63]:
# Train the model
for epoch in range(num_epochs):
    # Train the model for one epoch
    loss = train(model, lossfn, optimizer, X_train, y_train)

    # Print the loss every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}, percent_loss = {(loss/len(y_train))*100 :2f}")


Epoch [100/1000], Loss: 8.7145, percent_loss = 1.089314
Epoch [200/1000], Loss: 8.6377, percent_loss = 1.079719
Epoch [300/1000], Loss: 8.6520, percent_loss = 1.081500
Epoch [400/1000], Loss: 8.5611, percent_loss = 1.070138
Epoch [500/1000], Loss: 8.4437, percent_loss = 1.055462
Epoch [600/1000], Loss: 8.4692, percent_loss = 1.058649
Epoch [700/1000], Loss: 8.3843, percent_loss = 1.048031
Epoch [800/1000], Loss: 8.4530, percent_loss = 1.056628
Epoch [900/1000], Loss: 8.2550, percent_loss = 1.031878
Epoch [1000/1000], Loss: 8.1569, percent_loss = 1.019610


Test the model in evaluation mode or no_grad mode meaning the gradients are not calculated as it is not necessary.

In [64]:
# Test the model
with torch.no_grad():
    inputs = torch.from_numpy(c).float()
    labels = torch.from_numpy(d).float()
    y_preds = model(inputs)
    loss = lossfn(y_preds, labels)
    percent_loss = (loss / len(labels)) * 100
    print(f"Test Loss: {loss:.4f}, Percent_loss: {percent_loss:.2f}%")

Test Loss: 10.0646, Percent_loss: 5.03%


Do some predictions 

In [65]:
new_X = torch.tensor([[26.923264,54.880430,26.256636]], dtype=torch.float32)
new_y_pred = model(new_X)

In [66]:
print(new_y_pred)

tensor([[35.0747, 18.7170]], grad_fn=<AddmmBackward0>)


Pretty close!! End of project!